In [42]:
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt

In [43]:
import pandas as pd
import numpy as np
from keras.models import load_model

# Load the Keras model
model = load_model('BodyFatEstimator.h5')

# Load the dataset into a pandas DataFrame
dataFrame = pd.read_csv("bodyfat.csv")

# # Create a new row as a dictionary
# newRow = {
#     "Weight": 204.6, "Height": 72, "Neck": 35, "Chest": 98, "Abdomen": 84,
#     "Hip": 91, "Thigh": 55, "Knee": 38, "Ankle": 20, "Biceps": 32,
#     "Forearm": 27, "Wrist": 16, "BodyFat":16.4
# }
#
# # Append the new row to the DataFrame
# dataFrame = pd.concat([dataFrame, pd.DataFrame([newRow])], ignore_index=True)

print("Data set loaded. Num examples: ", len(dataFrame))

# Perform additional operations
dataFrame['BMI'] = (dataFrame['Weight'] * dataFrame['Weight']) / dataFrame['Height']
dataFrame['AbdomenNeckRatio'] = dataFrame['Abdomen'] / dataFrame['Neck']
dataFrame['ForearmWristRatio'] = dataFrame['Forearm'] / dataFrame['Wrist']
dataFrame['BicepsWristRatio'] = dataFrame['Biceps'] / dataFrame['Wrist']
dataFrame['ThighAnkleRatio'] = dataFrame['Thigh'] / dataFrame['Ankle']


print("Made training and test sets")

Data set loaded. Num examples:  252
Made training and test sets


In [44]:
inputs = {
    'BMI':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='BMI'),
    'AbdomenNeckRatio':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='AbdomenNeckRatio'),
    'BicepsWristRatio':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='BicepsWristRatio'),
    'Abdomen':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='Abdomen'),
    'ThighAnkleRatio':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='ThighAnkleRatio'),
    'Chest':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='Chest'),
    'Forearm':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='Forearm'),
    'Hip':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='Hip'),
    'Knee':
        tf.keras.layers.Input(shape=(1,), dtype=tf.float32,
                              name='Knee'),
}

# Useless Features:
# Abdomen

# Normalization for BMI
bmi = tf.keras.layers.Normalization(
    name='normalization_bmi',
    axis=None
)
bmi.adapt(dataFrame['BMI'])
bmi_input = tf.keras.Input(shape=(1,), name='BMI')
bmi_output = bmi(bmi_input)

# Normalization for AbdomenNeckRatio
abdomen_neck_ratio = tf.keras.layers.Normalization(
    name='normalization_abdomen_neck_ratio',
    axis=None
)
abdomen_neck_ratio.adapt(dataFrame['AbdomenNeckRatio'])
abdomen_neck_ratio_input = tf.keras.Input(shape=(1,), name='AbdomenNeckRatio')
abdomen_neck_ratio_output = abdomen_neck_ratio(abdomen_neck_ratio_input)

# Normalization for BicepsWristRatio
biceps_wrist_ratio = tf.keras.layers.Normalization(
    name='normalization_biceps_wrist_ratio',
    axis=None
)
biceps_wrist_ratio.adapt(dataFrame['BicepsWristRatio'])
biceps_wrist_ratio_input = tf.keras.Input(shape=(1,), name='BicepsWristRatio')
biceps_wrist_ratio_output = biceps_wrist_ratio(biceps_wrist_ratio_input)

# Concatenate our inputs into a single tensor.
preprocessing_layers = tf.keras.layers.Concatenate()(list(inputs.values()))

# Normalization for Abdomen
abdomen = tf.keras.layers.Normalization(
    name='normalization_abdomen',
    axis=None
)
abdomen.adapt(dataFrame['Abdomen'])
abdomen_input = tf.keras.Input(shape=(1,), name='Abdomen')
abdomen_output = abdomen(abdomen_input)

# Normalization for ThighAnkleRatio
thigh_ankle_ratio = tf.keras.layers.Normalization(
    name='normalization_thigh_ankle_ratio',
    axis=None
)
thigh_ankle_ratio.adapt(dataFrame['ThighAnkleRatio'])
thigh_ankle_ratio_input = tf.keras.Input(shape=(1,), name='ThighAnkleRatio')
thigh_ankle_ratio_output = thigh_ankle_ratio(thigh_ankle_ratio_input)

# Normalization for Chest
chest = tf.keras.layers.Normalization(
    name='normalization_chest',
    axis=None
)
chest.adapt(dataFrame['Chest'])
chest_input = tf.keras.Input(shape=(1,), name='Chest')
chest_output = chest(chest_input)

# Normalization for Forearm
forearm = tf.keras.layers.Normalization(
    name='normalization_forearm',
    axis=None
)
forearm.adapt(dataFrame['Forearm'])
forearm_input = tf.keras.Input(shape=(1,), name='Forearm')
forearm_output = forearm(forearm_input)

# Normalization for Hip
hip = tf.keras.layers.Normalization(
    name='normalization_hip',
    axis=None
)
hip.adapt(dataFrame['Hip'])
hip_input = tf.keras.Input(shape=(1,), name='Hip')
hip_output = hip(hip_input)

# Normalization for Knee
knee = tf.keras.layers.Normalization(
    name='normalization_knee',
    axis=None
)
knee.adapt(dataFrame['Knee'])
knee_input = tf.keras.Input(shape=(1,), name='Knee')
knee_output = knee(knee_input)

print("Preprocessing layers defined.")

Preprocessing layers defined.


In [45]:
# Create Normalization layers
train_bodyfat_normalized = tf.keras.layers.Normalization(axis=None)
train_bodyfat_normalized.adapt(
    np.array(dataFrame['BodyFat']))

test_bodyfat_normalized = tf.keras.layers.Normalization(axis=None)
test_bodyfat_normalized.adapt(
    np.array(dataFrame['BodyFat']))

In [49]:
# Create an empty DataFrame to store the results
result_table = pd.DataFrame(columns=['Row', 'Predicted Body Fat', 'Correct Body Fat', 'Difference +-'])

# Get the mean and variance from the normalization layer
mean = train_bodyfat_normalized.mean.numpy()
variance = train_bodyfat_normalized.variance.numpy()

print("mean " + str(mean))
print("variance " + str(variance))


# Iterate over the rows of the test dataframe
for index, row in dataFrame.iterrows():
    test_features = {name: np.array([value]) for name, value in row.items() if name != 'BodyFat'}

    # Get the correct label for the current row
    correct_label = np.array([row['BodyFat']])
    correct_label_normalized = (correct_label - mean) / np.sqrt(variance)

    # Make predictions using the model
    predictions = model.predict(test_features, verbose=0)

    predicted_bodyfat_normalized = predictions['dense_output'][0][0]
    predicted_bodyfat = (predicted_bodyfat_normalized * np.sqrt(variance)) + mean
    predicted_bodyfat = np.round(predicted_bodyfat, decimals=1)

    actual_bodyfat = np.round((correct_label_normalized * np.sqrt(variance)) + mean, decimals=1)
    difference = np.round(predicted_bodyfat - actual_bodyfat, 1)
    row_data = pd.DataFrame({'Row': [index],
                             'Predicted Body Fat': [predicted_bodyfat],
                             'Correct Body Fat': [actual_bodyfat],
                             'Difference +-': [difference]})

    # Concatenate the row DataFrame to the result_table
    result_table = pd.concat([result_table, row_data], ignore_index=True)

pd.options.display.max_rows = 100
result_table

mean [19.150793]
variance [69.757904]


,Row,Predicted Body Fat,Correct Body Fat,Difference +-
0,0,[16.3],[12.3],[4.0]
1,1,[11.4],[6.1],[5.3]
2,2,[18.9],[25.3],[-6.4]
3,3,[14.6],[10.4],[4.2]
4,4,[24.7],[28.7],[-4.0]
...,...,...,...,...
249,249,[27.3],[29.3],[-2.0]
250,250,[26.0],[26.0],[0.0]
251,251,[26.8],[31.9],[-5.1]
252,252,[nan],[16.4],[nan]
